In [20]:
from tensorflow import keras
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [21]:
model = keras.models.load_model("worked_example_model.h5")

In [22]:
housing = fetch_california_housing()

In [23]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [24]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)     #get fit parameters for scaling
X_valid = scaler.transform(X_valid)         # scale validation and test test using those parameters
X_test = scaler.transform(X_test)

# Using callbacks

In [25]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:,2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [26]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("worked_example_model.h5",
                                                save_best_only=True)
history = model.fit([X_train_A, X_train_B], [y_train, y_train], epochs=10,
                    validation_data = ([X_valid_A, X_valid_B], [y_valid, y_valid]),
                    callbacks=[checkpoint_cb])

Epoch 1/10
363/363 [==============================] - 1s 4ms/step - loss: 0.3516 - main_output_loss: 0.3396 - aux_output_loss: 0.4599 - val_loss: 0.3454 - val_main_output_loss: 0.3320 - val_aux_output_loss: 0.4666
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 0.3434 - main_output_loss: 0.3315 - aux_output_loss: 0.4505 - val_loss: 0.3383 - val_main_output_loss: 0.3248 - val_aux_output_loss: 0.4605
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.3416 - main_output_loss: 0.3302 - aux_output_loss: 0.4443 - val_loss: 0.3366 - val_main_output_loss: 0.3234 - val_aux_output_loss: 0.4555
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.3404 - main_output_loss: 0.3292 - aux_output_loss: 0.4408 - val_loss: 0.3326 - val_main_output_loss: 0.3197 - val_aux_output_loss: 0.4488
Epoch 5/10
363/363 [==============================] - 1s 3ms/step - loss: 0.3364 - main_output_loss: 0.3252 - aux_output_loss: 0.4370 - val_loss

In [27]:
#rollback to the best model
model = keras.models.load_model("worked_example_model.h5")

# Custom Callbacks

In [28]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):                     # displays ratio between valiation loss and training loss
    def on_epoch_end(self, epoch, logs):
        print('\nval/train: {:.2f}'.format(logs['val_loss'] / logs['loss']))

# Introduction to Tensorboard

In [29]:
# define root directory for tensorboard logs
import os
root_logdir = os.path.join(os.curdir, "my_logs")

In [30]:
def get_run_dir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [31]:
run_logdir = get_run_dir()

# Train model again to visualize Tensorboard results

In [32]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)     #get fit parameters for scaling
X_valid = scaler.transform(X_valid)         # scale validation and test test using those parameters
X_test = scaler.transform(X_test)

In [33]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation = 'relu', input_shape = X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_squared_error", optimizer = "sgd")

In [34]:
# Keras provides a Tensorboard callback
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train,
                    epochs = 20,
                    validation_data=(X_valid,y_valid),
                    callbacks=[tensorboard_cb]
)

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 1.0217 - val_loss: 0.5190
Epoch 2/20
363/363 [==============================] - 1s 2ms/step - loss: 0.5311 - val_loss: 0.4568
Epoch 3/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4853 - val_loss: 0.4456
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4655 - val_loss: 0.4487
Epoch 5/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4560 - val_loss: 0.4237
Epoch 6/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4510 - val_loss: 0.4187
Epoch 7/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4410 - val_loss: 0.4217
Epoch 8/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4323 - val_loss: 0.4175
Epoch 9/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4270 - val_loss: 0.4069
Epoch 10/20
363/363 [==============================] - 1s 2ms/step - loss: 0.4214 - val_loss: 0.4014

In [36]:
# Running Tensorboard
%load_ext tensorboard       #loads Tensorboard extension
%tensorboard --logdir=./my_logs --port=6006     #starts tensorboard server on port 6006

# now visit "http://localhost:6006" on your browser to access Tensorboard

In [39]:
# Create summarywriter to log scalars, histograms, images, audio, and text to visualize in tensorboard
import numpy as np
test_logdir = get_run_dir()
writer = tf.summary.create_file_writer(test_logdir)

In [41]:
with writer.as_default():
    for step in range(1, 1000+1):
        tf.summary.scalar("my_scalar", np.sin(step / 10), step=step)
        data = (np.random.randn(100) + 2) * step / 100
        tf.summary.histogram("my_hist", data, buckets=50, step=step)
        images = np.random.rand(2,32,32,3)
        tf.summary.image("my_images", images *step / 1000, step = step)
        texts = ["The step is " + str(step), "It's square is " + str(step**2)]
        tf.summary.text("my_text", texts, step=step)
        sine_wave = tf.math.sin(tf.range(12000) / 48000 *2 * np.pi * step)
        audio = tf.reshape(tf.cast(sine_wave, tf.float32), [1, -1,  1])
        tf.summary.audio("my_audio", audio, sample_rate=48000, step=step)